# Финальный проект: SQL

Компания купила крупный сервис для чтения книг по подписке. 

### Цель исследования
- Проанализировать базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг, с целью формирования ценностного предложения для продукта 

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### ER-диаграмма

![DB map](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)

In [1]:
# !pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import sql

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


#### Исследуем таблицы

In [3]:
# функция для вывода первых 5 строк

def query_top5(table_name):
    query = f'SELECT * FROM {table_name} LIMIT 5'
    display(pd.io.sql.read_sql(query, con = engine))

# выведем первые 5 строк таблиц
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query_top5(table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Задача 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
query = '''SELECT COUNT(DISTINCT book_id)
            FROM books 
            WHERE CAST(publication_date as date) > '2000-01-01'
            '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


- После 1 января 2000 года вышло 819 книг

#### Задача 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [6]:
query = '''WITH
        a AS (SELECT book_id,
                    COUNT(review_id) AS reviews_count
            FROM reviews
            GROUP BY book_id),
        b AS (SELECT book_id,
                    AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id)
        
        SELECT b.book_id,
                a.reviews_count,
                ROUND(b.avg_rating,2) as avg_rating
        FROM a
        FULL JOIN b ON a.book_id=b.book_id
        ORDER BY book_id
            '''
pd.io.sql.read_sql(query, con = engine)


,book_id,reviews_count,avg_rating
0,1,2.0,3.67
1,2,1.0,2.50
2,3,3.0,4.67
3,4,2.0,4.50
4,5,4.0,4.00
...,...,...,...
995,996,3.0,3.67
996,997,3.0,3.40
997,998,4.0,3.20
998,999,2.0,4.50


#### Задача 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [7]:
query = '''WITH 
        a AS (SELECT publisher_id,
                COUNT(num_pages) as books_over_50_count
            FROM books
            WHERE num_pages > 50
            GROUP BY publisher_id
            ORDER BY books_over_50_count DESC
            LIMIT 1)
        SELECT a.publisher_id,
                publishers.publisher,
                a.books_over_50_count
        FROM a
        LEFT JOIN publishers on a.publisher_id=publishers.publisher_id
            '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,books_over_50_count
0,212,Penguin Books,42


- Издательство Penguin Books выпустило 42 книги толще 50 страниц

#### Задача 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [24]:
query='''
WITH 
a AS (SELECT book_id,
        COUNT(rating) AS rating_count
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating)>=50),
b AS (SELECT books.author_id as author_id,
            AVG(rating) AS avg_rating
      FROM a
      INNER JOIN books on a.book_id=books.book_id
      LEFT JOIN ratings on a.book_id=ratings.book_id
      GROUP BY author_id
      ORDER BY avg_rating DESC
      LIMIT 1)
      
SELECT b.author_id,
        authors.author,
        ROUND(b.avg_rating,2) AS avg_rating
FROM b
LEFT JOIN authors on b.author_id=authors.author_id
      
'''
pd.io.sql.read_sql(query, con = engine)



,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


- Автором с самой высокой средней оценкой книг (среди книг с более 50 оценками) является J.K. Rowling/Mary GrandPré - и средняя оценка автора составляет 4.29

#### Задача 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query = '''WITH
            a AS (SELECT username,
                        COUNT(rating) AS rating_count
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating)>50),
            b AS (SELECT username,
                        COUNT(review_id) AS reviews_count
                FROM reviews
                GROUP BY username)
            
            SELECT ROUND(AVG(b.reviews_count),1)
            FROM a
            LEFT JOIN b on a.username=b.username
            '''
pd.io.sql.read_sql(query, con = engine)

- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24.3